In [1]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
#importação da tabela
import duckdb

# Correcting the SQL query syntax for the IN clause (using parentheses instead of curly braces)
query = """
SELECT *
FROM delta_scan('/lakehouse/default/Tables/comentarios_copacabana')
LIMIT 20
"""

# Executing the query and displaying the result
df = duckdb.sql(query).df()

In [3]:
#definição do openai
import openai

prompt_definition = [
    {"role": "system", 
    "content": 
    """
    Você deve avaliar os comentarios feitos por visitantes a atrações turísticas, traga a analise de sentimento de -1 a 1 e informe se foi uma opinião boa  ou ruim sempre seprando a nota e o resultado da avaliação com;
    """
    }
    ]
conversation_history = prompt_definition
# Configuração do modelo
DEPLOYMENT_ID = 'gpt-4o' 

def chat_with_bot(user_input):
    # Adiciona a mensagem do usuário ao histórico
    conversation_history.append({"role": "user", "content": user_input})
    
    # Faz a chamada à API
    response = openai.ChatCompletion.create(
        deployment_id=DEPLOYMENT_ID,
        messages=conversation_history,
        temperature=0.1
    )
    
    # Obtém a resposta do assistente
    assistant_reply = response.choices[0].message["content"]
    
    # Adiciona a resposta do assistente ao histórico
    conversation_history.append({"role": "assistant", "content": assistant_reply})
    
    return assistant_reply

def convesation(input_text):
    user_input = input_text
    response = chat_with_bot(user_input)
    return f'{response}'


In [5]:
#codigo de loop para enrequecimento do openai

import pandas as pd


avaliations = []

for i in range(len(df)):
    response_chat = convesation(df['comentario'][i])
    avaliations.append(response_chat)
    conversation_history = prompt_definition

# Creating a new DataFrame with the evaluations and opportunity IDs
df['avaliacoes'] = avaliations


In [ ]:
#separando os resultados do openai

df[['indice_sentimento', 'avaliacao_comentario']] = df['avaliacoes'].str.split(';', expand=True)

# Converte o índice de sentimento para float
df['indice_sentimento'] = df['indice_sentimento'].astype(float)

df['comentario'] = df['comentario'].str[:250]

# (Opcional) Remove a coluna original se não precisar mais
df = df.drop(columns=['avaliacoes'])


In [9]:
#embedendo report de PBI no notebook do Fabric
from powerbiclient import QuickVisualize, get_dataset_config

# Create a Power BI report object from your data
PBI_visualize = QuickVisualize(get_dataset_config(df))

# Render new report
PBI_visualize

QuickVisualize()